# Water bodies detection sub-workflow

Invoke a Common Workflow Language Workflow for detecting water bodies

This notebook is linked to: https://eoap.github.io/zarr-cloud-native-format/sub-workflows/water-bodies-detection/



## Setup

In [1]:
export WORKSPACE=/workspace/zarr-cloud-native-format
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

curl -q -L https://github.com/eoap/zarr-cloud-native-format/releases/download/0.5.0/app-water-bodies.0.5.0.cwl > ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl 2> /dev/null

## Run the water bodies detection sub-workflow step

Inspect and use `cwltool` to run the water bodies detection sub-workflow definition:


In [2]:
cat ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl | yq e '.["$graph"][2]' -

class: Workflow
id: detect_water_body
label: Water body detection based on NDWI and otsu threshold
doc: Water body detection based on NDWI and otsu threshold
requirements:
  - class: ScatterFeatureRequirement
inputs:
  aoi:
    doc: area of interest as a bounding box
    type: string
  epsg:
    doc: EPSG code
    type: string
    default: "EPSG:4326"
  bands:
    doc: bands used for the NDWI
    type: string[]
  item:
    doc: STAC item
    type: string
outputs:
  - id: detected_water_body
    outputSource:
      - otsu/binary_mask_item
    type: File
  - id: ndwi
    outputSource:
      - normalized_difference/ndwi
    type: File
steps:
  crop:
    run: "#crop"
    in:
      item: item
      aoi: aoi
      epsg: epsg
      band: bands
    out:
      - cropped
    scatter: band
    scatterMethod: dotproduct
  normalized_difference:
    run: "#norm_diff"
    in:
      rasters:
        source: crop/cropped
    out:
      - ndwi
  otsu:
    run: "#otsu"
    in:
      raster:
        sour

Run the CWL description, but first prepare the parameters.

The previous step generated part of inputs for the water bodies detection sub-workflow:

In [3]:
cat convert-search-results.json

{
    "aoi": "-121.399,39.834,-120.74,40.472",
    "items": [
        "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2A_10TFK_20210728_0_L2A",
        "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2A_10TFK_20210728_1_L2A",
        "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2B_10TFK_20210723_1_L2A",
        "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2B_10TFK_20210723_0_L2A",
        "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2A_10TFK_20210718_0_L2A",
        "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2B_10TFK_20210713_1_L2A",
        "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2B_10TFK_20210713_0_L2A",
        "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2A_10TFK_20210708_0_L2A",
        "https://earth-search.aws.element84.com/v1

Let's add the `bands` and `epsg`:

In [4]:
cat convert-search-results.json | jq .aoi

"-121.399,39.834,-120.74,40.472"


In [5]:
aoi=$( cat convert-search-results.json | jq '.aoi' )

echo ${aoi}

"-121.399,39.834,-120.74,40.472"


In [6]:
item=$( cat convert-search-results.json | jq '.items[0]' )

echo ${item}

"https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2A_10TFK_20210728_0_L2A"


Let's create the parameter file:

In [7]:
cat <<EOF > water-bodies-params.yaml
item: ${item}
aoi: ${aoi}
bands:
  - green
  - nir
epsg: "4326"
EOF

cat water-bodies-params.yaml | yq .

item: "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2A_10TFK_20210728_0_L2A"
aoi: "-121.399,39.834,-120.74,40.472"
bands:
  - green
  - nir
epsg: "4326"


In [8]:


cwltool \
    --podman \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl#detect_water_body \
    water-bodies-params.yaml > water-bodies-results.json 2> water-bodies.log

Let's look at the content of the stderr:

In [9]:
cat water-bodies.log | egrep -v "WARNING|JSHINT"

INFO /home/fbrito/.local/bin/cwltool 3.1.20250110105449
INFO Resolved '/workspace/zarr-cloud-native-format/cwl-workflow/app-water-bodies.cwl#detect_water_body' to 'file:///workspace/zarr-cloud-native-format/cwl-workflow/app-water-bodies.cwl#detect_water_body'
INFO [workflow ] start
INFO [workflow ] starting step crop
INFO [step crop] start
INFO [job crop] /tmp/qf0p_sxm$ podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/tmp/qf0p_sxm,target=/wnSJRC \
    --mount=type=bind,source=/tmp/jweghpud,target=/tmp \
    --workdir=/wnSJRC \
    --read-only=true \
    --user=1000:1000 \
    --rm \
    --cidfile=/tmp/r2d8uvy4/20260115112514-901893.cid \
    --env=TMPDIR=/tmp \
    --env=HOME=/wnSJRC \
    --env=PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin \
    --env=PYTHONPATH=/app \
    ghcr.io/eoap/mastering-app-package/crop@sha256:25985f43b299e12b8c476746c4b5edf328f4d0b0fea9d017c3dafd2f4f5116ba \
    python \
    -m \
    app \
    --aoi \
    -

Let's inspect the stdout produced. There are two `output` blocks with the id `aoi` and `items`. 

These are what the water bodies detection sub-workflow expects as inputs

In [10]:
cat water-bodies-results.json | jq . -

{
  "detected_water_body": {
    "location": "file:///workspace/zarr-cloud-native-format/runs/otsu.tif",
    "basename": "otsu.tif",
    "class": "File",
    "checksum": "sha1$a7f9a22f096d7bb5e2a0ecfe0abba2eab2350f1f",
    "size": 1100785,
    "path": "/workspace/zarr-cloud-native-format/runs/otsu.tif"
  },
  "ndwi": {
    "location": "file:///workspace/zarr-cloud-native-format/runs/norm_diff.tif",
    "basename": "norm_diff.tif",
    "class": "File",
    "checksum": "sha1$9e77047b2de97a3c2bf78e18e29867463c418c53",
    "size": 233064610,
    "path": "/workspace/zarr-cloud-native-format/runs/norm_diff.tif"
  }
}
